# Tritium desorption in fusion reactor walls

In this example, we look at the ability of `twinLab` to model the [desorption](https://en.wikipedia.org/wiki/Desorption) (the physical process where a previously adsorbed substance is released from a surface) of tritium (a radioactive isotope of hydrogen) in the wall of a nuclear fusion reactor. Fusion generates almost no radioactive waste, and the little waste that it does produce has a short half-life. However, the interior of the reactor wall is bombarded by a high-neutron flux during fusion, far higher than any naturally-occurring radioactive process, and therefore the properties of materials under high neutron bombardment are unknown. Computer simulations are required to understand the material properties in such extreme circumstances, but simulations are expensive in terms of computational power, and cannot be run at every point in parameter space under consideration. `twinLab` can be used to train simulation surrogate models using data from a sparse array of simulations. This allows for meaningful interpolation and extrapolation to unexplored regions of parameter space, together with a calibrated uncertainty estimate on the accuracy of the simulation surrogate.

First, import the required libraries

In [ ]:
# Standard imports
import os
from pprint import pprint

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Project imports
import twinlab as tl

Now, define some parameters. We need to provide `twinLab` with locations of data files (in `.csv` format) and directories, together with a chosen surrogate campaign name (here `ukaea`) and a list of the input parameters that we are going to use as inputs to our model. Here these 5 parameters are $E_1$, $E_2$, $E_3$, $n_1$, and $n_2$ and quantify properties of the material that can trap tritium nuclei. The $E_i$ parameters are energies while the $n_i$ parameters are particle number densities. $E_1$ and $n_1$ pertain to the same trap, as do $E_2$ and $n_2$, but in the model parameterized by $E_3$ the value of $n_3$ is calculated internally, and evolves with time, and therefore this is not an input parameter for our training regime.

In [ ]:
# Files and file paths
file_train = os.path.join("resources", "datasets", "ukaea.csv")
campaign_dir = os.path.join("resources", "campaigns", "ukaea")
file_grid = os.path.join(campaign_dir, "grid.csv")
# file_eval = campaign_dir + "eval.csv"
# file_eval = campaign_dir + "post.csv"
file_eval = os.path.join(campaign_dir, "test.csv")

# Parameters
dataset_id = "fusion"
campaign_id = dataset_id
inputs = ["E1", "E2", "E3", "n1", "n2"]

Load the necessary data from `.csv` files for training and plotting

In [ ]:
df_train = pd.read_csv(file_train)
df_eval = pd.read_csv(file_eval)
df_grid = pd.read_csv(file_grid, header=None)
display(df_train)

Next, we create the parameter dictionary that we need to give to run twinLab.  At a minimum the user must provide the `filename` of the dataset on which we want to train our model (`.csv` format), together with the columns that we will take to be the `inputs` and `outputs` of our model, once that is trained. By default, `twinLab` will use all of the data for training. This latter choice can be overridden by adding e.g., `train_test_split=1000` so that only the first 1000 entries in `filename` are used for training, and the remaining examples are then used for internal model evaluation.

In this case we are training a functional model, which means that we want to return a *function* at every point in parameter space ($E_1$, $E_2$, $E_3$, $n_1$, $n_2$). In this case, our function describes the tritium desorption rate, $D$, of the material of the reactor wall as a function of temperature (rate of emitted nuclei per reactor wall area). The training of the surrogate is agnostic to the values of reactor temperature, $T$, so we must provide this by hand (`df_grid` above, from `file_grid`). The output of our model will therefore be the function $D(T; E_1, E_2, E_3, n_1, n_2)$.

The `twinLab` model achieves this by predicting the value of $D$ at $\sim 500$ points in $T$ in a regularly-spaced grid between $300\mathrm{K}$ and $800\mathrm{K}$. The correlations between points adjacent in $T$ are incorporated naturally by the model, and `twinLab` provides a model uncertainty. Here we call the outputs `y`, rather than `D`, as per the typical data-science convention.

In [ ]:
# Column headings for outputs
y_outputs = [f"y{i}" for i in range(len(df_grid))]

# Parameters
params = {
    "dataset_id": dataset_id,
    "inputs": inputs,
    "outputs": y_outputs,
    "decompose_outputs": True,
    "train_test_ratio": 0.25,
}

Now the dataset can be uploaded to the cloud

In [ ]:
tl.upload_dataset(file_train, dataset_id, verbose=True)

List all of the datasets under our username to check that the upload worked

In [ ]:
_ = tl.list_datasets(verbose=True)

Print some useful properties of the dataset

In [ ]:
query = tl.query_dataset(dataset_id, verbose=True)
pprint(query)

Train the surrogate model, this step takes around 6 minutes.

In [ ]:
tl.train_campaign(params, campaign_id, verbose=True)

List the campaigns again to ensure that training has been completed

In [ ]:
_ = tl.list_campaigns(verbose=True)

Query the campaign to check how training has proceeded

In [ ]:
_ = tl.query_campaign(campaign_id, verbose=True)

Evaluate the trained emulator on `X` ($E_1, E_2, E_3, n_1, n_2$) from the evaluation file

In [ ]:
df_mean, df_std = tl.predict_campaign(file_eval, campaign_id, verbose=False)
display(df_mean)
display(df_std)

Plot the results for a few different `y` values to sanity check. These values correspond to the tritium desorption flux at different reactor temperatures: $D(T_i)$.

In [ ]:
# Parameters for plot
color = "blue"
alpha = 0.8
xs = {"E1": r"$E_{1}$", "E2": r"$E_{2}$", "E3": r"$E_{3}$", "n1": r"$n_{1}$", "n2": r"$n_{2}$"}
ys = {f"y{i}": fr"$y_{{{i}}}$" for i in [0, 100, 150]}

# Plot some examples
nrow, ncol = len(ys), len(xs)
plt.subplots(nrow, ncol, figsize=(25, 10))
nplot = 0
for y, y_label in ys.items():
    for x, x_label in xs.items():
        nplot += 1
        plt.subplot(nrow, ncol, nplot)
        plt.errorbar(df_eval[x], df_mean[y], yerr=df_std[y], marker='.', lw=1, ls='None', color=color, alpha=alpha, label="Model")
        plt.plot(df_train[x], df_train[y], ".", color="black", alpha=0.1, label="Training data")
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        if nplot==1: plt.legend()
plt.show()

We are now in a position to plot the output of the trained surrogate model together with the "truth" from the evaluation file. The intensity of the blue regions correspond to the probability as predicted by the surrogate. It can be seen that the truth mainly goes through the regions of high probability as predicted by the model. Thus we have trained an accurate surrogate for the tritium desorption process. This training was done on the cloud and the surrogate has been stored and can be evaluated extremely quickly, with uncertainty predictions built into the modelling.

In [ ]:
# Parameters for plot
error_inflation_factor = 1. # Factor to multiply error by for plotting
y_fac = 18 # Factor to divide y by for plotting [log10]
plot_eval = True
data_alpha = 0.75
plot_model_mean = True
plot_model_bands = True
plot_model_blur = False
nsigs = [1, 2]
model_alpha = 0.5
n_model_blur = 100
model_color = 'blue'
number_of_training_examples = 0
number_of_model_examples = 5

# Plot results
grid = df_grid.iloc[:, 0]
plt.subplots()
if (plot_model_blur or plot_model_bands) and not plot_model_mean: 
    plt.fill_between(grid, np.nan, np.nan, color=model_color, alpha=model_alpha, lw=0., label="Model predictions")
for example in range(number_of_training_examples): # Training examples
    train = df_train[y_outputs].iloc[example]/10**y_fac
    label = "Example training data" if example==0 else None
    plt.plot(grid, train, color='black', alpha=0.5, label=label)
for example in range(number_of_model_examples): # Model predictions
    mean = df_mean[y_outputs].iloc[example]/10**y_fac
    err = error_inflation_factor*df_std[y_outputs].iloc[example]/10**y_fac
    if plot_eval and (file_eval == campaign_dir + "test.csv"):
        eval = df_eval[y_outputs].iloc[example]/10**y_fac
        label = "Test data" if example==0 else None
        plt.plot(grid, eval, color='black', alpha=data_alpha, label=label)
    if plot_model_bands:
        for isig, nsig in enumerate(nsigs):
            label = "Model predictions" if (isig == 0) and (example == 0) else None
            plt.fill_between(grid, mean-nsig*err, mean+nsig*err, color=model_color, alpha=model_alpha/(isig+1), lw=0., label=label)
    if plot_model_blur and not plot_model_bands:
        alpha = tl.get_blur_alpha(n_model_blur, model_alpha)
        dys = tl.get_blur_boundaries(n_model_blur)
        for iy, dy in enumerate(dys):
            label = "Model predictions" if (iy == 0) and (example == 0) else None
            plt.fill_between(grid, mean-dy*err, mean+dy*err, color=model_color, alpha=alpha, lw=0., label=label)
    if plot_model_mean:
        label = "Model predictions" if (example==0) and (not plot_model_bands) and (not plot_model_blur) else None
        plt.plot(grid, mean, color=model_color, label=label, alpha=model_alpha)
plt.xlabel(r'Temperature [K]')
plt.xlim((grid.min(), grid.max()))
plt.ylabel(rf"Desorption rate [$10^{{{y_fac}}}$ $m^{{{-2}}}$ $s^{{{-1}}}$]")
plt.ylim(bottom=0.)
plt.legend()
plt.show()

Finally, delete the campaign if necessary...

In [ ]:
tl.delete_campaign(campaign_id, verbose=True)

... and delete the dataset if necessary.

In [ ]:
tl.delete_dataset(dataset_id, verbose=True)